## Mobile Robotics Report

**Group Members**: Luca Engel, Marc Nassif,  Giuseppe De Carlo,  Giada Ehrlich



## Table of contents
1. [Introduction](#introduction)
2. [Setup](#setup)
3. [Implementation](#implementation)
      - 3.1 [Vision](#vision)
         -  [Marker Detection](#marker-detection)
         -  [Grayscale Conversion and Thresholding](#thresholding)
         - [2D Grid Representation](#grid-representation)
         - [Dynamic Updates](#dynamic-updates)
      - 3.2 [Global Navigation](#global-navigation)
         -  [Dijkstra Algorithm](#dijkstra-algorithm)
         -  [Path Discretization](#path-discretization)
         -  [Integration with Vision System](#integration-vision)
      - 3.3 [Motion Control](#motion-control)
         -  [Direction calculation](#direction-calculation)
         -  [Rotate](#rotate)
         -  [Move](#move)
         -  [Movement Schematic](#movement-schematic)
         -  [Orientation Angle Challenges](#orientation-angle-challenges)
      - 3.4 [Local Navigation](#local-navigation)
         - [Judge Severity](#judge-severity)
         - [Rotation](#rotation)
         - [Circling](#circling)
         - [Global Navigation Resumption](#global-navigation-resumption)
      - 3.5 [Kalman Filter](#kalman-filter)
         - [Wheel Speed Measurements](#wheel-speed-measurements)
         - [Prediction and Update Step](#prediction-update-step)
         - [Handling Orientation Jumps](#handling-orientation-jumps)
         - [Adaptability to Vision Feed Cut-Off](#adaptability-vision)
         
4. [Results](#results)
   - 4.1 [Video](#video)
   - 4.2 [Run Demo](#demo)
   
5. [References](#references)
 


   







<!-- headings -->

<a id="introduction"></a>
## 1. Introduction




In mobile robotics, fusing computer vision and sensor-based feedback is essential to establish reliable global navigation algorithms, motion control strategies, and sensor-based local obstacle avoidance to enable robots to intelligently and reliably navigate through dynamic environments. This report describes our effort to develop a solution that enables the Thymio robot to traverse a terrain containing obstacles and to reach a goal.

Our project integrates multiple technologies to improve the navigation precision. With the help of a video feed provided by a camera, computer vision is used to detect the map, the obstacles in the map, the Thymio robot, and the goal. Based on this information, a 2d grid of cells is constructed. The global navigation aspect employs the Dijkstra algorithm to calculate the shortest path from the Thymio to the goal, accounting for dynamically changing events, such as kidnapping where the Thymio or the goal are displaced. The motion control mechanism guides the Thymio along the calculated path which is discretized into intermediate points, facilitating a smooth and precise traversal. For local obstacle avoidance, the Thymio relies on its proximity sensors, allowing it to detect and navigate around obstacles not identified by the computer vision part. This ensures robust navigation, even when visual data is limited. The implementation of the Kalman filter for position and orientation estimation plays a pivotal role in this project. The filter combines information from the computer vision part and the wheel speeds of the Thymio to improve the navigation precision. The filter is even capable of functioning when the camera feed is obscured for a while, showing the robustness of our system.

This report details the design, implementation, and performance of our solution and shows how the combination of sensor data can drastically improve the capabilities and precision in robotic navigation.

<a id="setup"></a>
## 2. Setup 


### Libraries required: 
- NumPy
- tmdclient
- OpenCV
- FilterPy

### Method overview: 

The program initially creates a global map after processing video frames acquired through the webcam. It subsequently extracts a path through global navigation and adheres to this route unless faced with kidnapping or local navigation triggers.

The provided schema illustrates the code structure implemented for this purpose. In cases where the camera becomes obscured, thanks to the Kalman Filter, the robot continues to follow the pre-defined global path planning based on the wheel speeds until the camera sensing functionality is restored.

<div align="center">
    <img src="./report_images/implementation_structure1.png" alt="Image Alt Text" width="400" height="400"/>
    <em>Implementation Strucure</em>

####  Environment


Webcam  : AUKEY 1080P

Map :
- White background 
- Global obstacles: black 
- Local obstacles: small 3-dimensional objects
- Markers: ArUco Markers



<div align="center">
    <img src="./report_images/set_image.png" alt="Image Alt Text" width="300" height="400"/>
    <em>Environment Setup</em>
</div>

<a id="implementation"></a>
## 3. Implementation

<a id="vision"></a>
### 3.1 Vision


Our vision system forms the baseline of the Thymio's understanding of the environment. Using the OpenCV library, the map is created in multiple steps, each of which extracting information from the camera feed, allowing our program to identify markers, determine their orientation, and recognize obstacles. In the following, our procedure will be shown with the example of the following image feed:

<div align="center">
    <img src="./report_images/image_processing_images/simple_camera_feed.png" alt="Simple Camera Feed">
    <em>Unprocessed Camera Feed</em>
</div>

<a id="marker-detection"></a>
##### Marker Detection
We used OpenCV's ArUco marker detection algorithm to find the key elements and their orientation within the terrain, specifically, the Thymio robot, the goal, and the corners of the map. The following code snipped, taken from the ```ArUcoMarkerDetector``` class illustrate how the markers are searched for in the input ```frame``` coming from the video feed. Once the markers are detected, our algorithm also extracts theyr orientation. This is used to find the orientation of the Thymio robot for navigation purposes.

```Python
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

parameters = aruco.DetectorParameters()
parameters.minMarkerPerimeterRate = MARKER_MIN_PERIMETER_RATE
# corners go clockwise from top left
corners, ids, rejected_img_points = aruco.detectMarkers(gray, self.marker_dict, parameters=parameters)

ids_to_direction = {}

# [This part extracts the direction the markers are pointing to]
if ids is not None:
    corners = np.array(corners)
    ids = np.array(ids)
    centroids = []
    for id in range(len(ids)):
        top_left = corners[id][0][0]
        bottom_left = corners[id][0][3]
        top_middle = 1 / 2 * (corners[id][0][0] + corners[id][0][1])

        # Calculate centroid (middle) of the marker
        centroid = np.mean(corners[id][0], axis=0)
        cv2.circle(frame, tuple(map(int, centroid)), 5, (0, 255, 0), -1)

        direction = top_middle - centroid
        ids_to_direction[id] = direction

        cv2.arrowedLine(frame, tuple(map(int, centroid)),
                        tuple(map(int, top_middle + direction)), (0, 255, 0), thickness=2, tipLength=0.1)

frame_markers = aruco.drawDetectedMarkers(frame.copy(), corners, ids)
```

Based on this information, a transformation matrix is applied to the image to place the map corners into the corners of the image so that the map fills the entire video feed. The perspective transform is computed by OpenCV's ```getPerspectiveTransform``` function based on four $(x, y)$ tuples and their corresponding ```destination_corners``` tuples. In our case, the four input tuples are the respective outermost corner of each of the ArUco markers delimiting the map and the destination tuples are the corner locations of our grid.

```Python
sorted_marker_positions = [marker_for_corner[i] for i in range(4)]
new_corners = np.array(sorted_marker_positions)

# Perspective transformation to adjust the image to have the markers in the corners
destination_corners = np.array([[0, 0], [w - 1, 0], [w - 1, h - 1], [0, h - 1]], dtype=np.float32)

self.transformation_matrix = cv2.getPerspectiveTransform(np.float32(new_corners), destination_corners)
base_frame, corners, processed_frame = self._transform_images(base_frame, corners, frame_markers, h, processed_frame, w)
```

Once the transformation matrix is computed our ```_transform_images``` function applies this transformation to the image feed and the elements displayed on it with OpenCV's ```warpPerspective``` function as follows:
```Python
def _transform_images(self, base_frame, corners, frame_markers, h, processed_frame, w):
    """
    Transform the base frame and the frame with markers drawn on it to place the corner markers in the frame corners.
    :param base_frame: base frame
    :param corners: markers' corners
    :param frame_markers: frame with markers drawn on it
    :param h: height of the frame
    :param processed_frame: frame to be transformed
    :param w: width of the frame
    :return: base frame, transformed corners, transformed frame with markers drawn on it
    """
    processed_frame = cv2.warpPerspective(frame_markers, self.transformation_matrix, (w, h))
    base_frame = cv2.warpPerspective(base_frame, self.transformation_matrix, (w, h))
    corners = cv2.perspectiveTransform(corners.reshape(-1, 1, 2), self.transformation_matrix).reshape(corners.shape)
    return base_frame, corners, processed_frame
    
```
The resulting processed image containing the framed ArUco markers and their orientation arrows is displayed below.

<div align="center">
    <img src="./report_images/image_processing_images/normal_feed_img.png" alt="Processed Image with Map Filling the Entire Screen">
    <em>Processed Feed with Map Filling the Screen</em>
</div>

<a id="thresholding"></a>
##### Grayscale Conversion and Thresholding
To simplify the object detection, we converted the camera feed to grayscale. As the objects are represented as black shaped, applying thresholding to the image allowed for easy extraction of their location in the environment. The following code is responsible for extracting the binary image which is also displayed below.

```Python
def _extract_black_objects(self, frame):
    """
    Extract the black objects from a frame.
    :param frame: The frame to extract the black objects from.
    :return: The binary image with black objects on a white background.
    """
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Threshold the image to get binary image with black objects on white background
    # binary image is 0 where no black object is detected, 255 where black object is detected
    _, binary_image = cv2.threshold(gray_frame, OBJECT_THRESHOLD, THRESHOLD_MAX_VAL, cv2.THRESH_BINARY_INV)

    return binary_image 
```

<div align="center">
    <img src="./report_images/image_processing_images/binary_img.png" alt="Binary Image">
    <em>Binary Image</em>
</div>

The resulting image still contains some noise due to the black thymio sensors being detected as obstacles. This can be observed in the top left corner and is resolved in the next step. 

<a id="grid-representation"></a>
##### 2D Grid Representation
Based on the information obtained in the previous steps, a 2D grid of cells is created. Each cell stores information about its status, indicating whether it is free (marked in white), it is occupied by the Thymio (marked in green), the goal (marked in blue), or an obstacle (marked in black). 

After conversion, the 2D grid looks as follows:

<div align="center">
    <img src="./report_images/image_processing_images/grid_map_with_islands_not_increased_object_size.png" alt="Grid Map with obstacles">
    <em>Grid Map with obstacles</em>
</div>



Here, still, the map contains the noise from the black sensors of the Thymio and the ArUco markers of which some are not entirely erased from the map. This is solved by removing all obstacles that are detected to be small islands. For a given pixel with coordinates $(x, y)$, the check to see if it is an island with a radius smaller or equal to ```radius``` is done as follows:
```Python
def _check_if_cells_are_island(self, x, y, radius):
    """
    Checks if the cells around the given cell island of size radius is an island
    :param x: the x coordinate of the cell
    :param y: the y coordinate of the cell
    :param radius: the radius of the island
    :return: True if the cells within the radius form an island, False otherwise
    """
    count = 0
    for k in range(-radius, radius + 1):
        for l in range(-radius, radius + 1):
            if (0 <= y + k < self._height
                    and 0 <= x + l < self._width
                    and not (-radius < k < radius and -radius < l < radius)):
                if self._grid[y + k, x + l] == CellType.OBJECT:
                    count += 1
    return count < 2  # i.e., no neighbours of the island within the radius are objects -> island
```

This pass through the map results in the following image with the noise removed:

<div align="center">
    <img src="./report_images/image_processing_images/grid_map_islands_removed_not_increased.png" alt="Grid Map with Islands Removed">
    <em>Grid Map with Islands Removed</em>
</div>




Lastly, to account for the width of the Thymio, obstacles are artificially enlarged to ensure that the Thymio cannot drive too close to the obstacles. In the path planning, the enlargement is considered as an obstacle which leads to the Thymio's path planning respecting the needed distance from the obstacles. The resulting map looks as follows:

<div align="center">
    <img src="./report_images/image_processing_images/grid_map_islands_removed_objects_increased.png" alt="[Grid Map with Object Size Increased">
    <em>Grid Map with Object Size Increased</em>
</div>



<a id="dynamic-updates"></a>
##### Dynamic Updates
The grid is continuously updated with the positions of the Thymio and the goal within the map. These updates ensure that the Thymio can make informed decisions and enable recomputing the path after kidnapping events or motion control adjustments. The obstacles themselves are never recomputed, allowing for local obstacles to be put down and remain undetected to the camera and to any Global Navigation paths generated.

This part lays the groundwork for successful navigation of the Thymio robot in the terrain.

<a id="global-navigation"></a>
### 3.2 Global Navigation



Our global navigation system guides the Thymio robot throught the map by calculating the shortest path from its start location to the goal. This path is found with the Dijkstra algorithm. Similar to the vision, the navigation is also constructed in a multi-step approach.

<a id="dijkstra-algorithm"></a>
##### Dijkstra Algorithm
The shortest path is calculated using the [Dijkstra algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm). The resulting path ensures that the Thymio can navigate through the environment while avoiding obstacles and ensures that it reaches the goal with the shortest path.

<a id="path-discretization"></a>
##### Path Discretization
To have the Thymio drive more smoothly, the computed path is discretized into a series of fewer cells. Each cell represents a direction change in the path. The Thymio receives these cells as intermediate goals allowing a more natural movement along the path and decreasing the number of sudden direction changes. The resulting path is marked red in the map and the direction changes purple. In this example, the Thymio robot's path would start at the top left at the green cell and end at the blue cell at the bottom right. With this addition, the grid looks as follows:

<div align="center">
    <img src="./report_images/image_processing_images/grid_map_with_path.png" alt="[Grid Map Marked with Path and Direction Changes">
    <em>Grid Map Marked with Path and Direction Changes</em>
</div>


<a id="integration-vision"></a>
##### Integration with Vision System
Based on the information received from the vision system, the path is calculated and obstacles can be avoided. Additionally, kidnapping of the thymio and the goal can be handled. Whenever such a kidnapping occurs, the path is recomputed so that the goal can be reached.


The combination of the Dijkstra algorithm, path discretization, and intermediate goal navigation, ensures a smooth and fast traversal of the map, even accounting for sudden events such as kidnapping.

<a id="motion-control"></a>
### 3.3 Motion Control

The motion control is designed to manage the speed and the direction of the Thymio. It was defined as a class to perform the basic movements of the robot. The class is used during the entire process, except when the Thymio needs to avoid an obstacle, and in this case, the *Local Navigation* becomes active.
 
The motion control handles 3 states of the robot's movement:
* **STOP**: Halts the robot to move during the process or when it reaches the goal.
* **ROTATE**: Before moving to the next point, the Thymio aligns its direction to face towards it. Once done, the robot transits to the  MOVE state.
* **MOVE**: Once the Thymio is aligned, it will begin moving towards the next point. This process is regulated by a PI controller to adjust the Thymio's trajectory.

Follow this, we will explain how we calculate the Thymio's direction and the desired direction. Afterwards, we will describe the main movement of our robot, the ROTATE and MOVE states, and describe how they operate.

<a id="direction-calculation"></a>    
##### Direction calculation
Before assigning a movement instruction to the Thymio, we must calculate its direction and the direction to the next point in its path because our PI controller uses the error between these two directions. Therefore, we implemented a function called `rotation_nextpoint` in `MotionControl.py` that receives these two direction vectors, which are detertmined in the function `get_thymio_and_path_directions` in `GlobalNavigation.py`. The function uses `atan2` from the library `math` to calculate the angle of each direction.


```python
def rotation_nextpoint(direction):
    """
        Rotation of the Thymio before moving to
        the next point
        Return the value in degree
    """
    return math.atan2(direction[0], direction[1]) / math.pi * 180
```
  
<a id="rotate"></a>   
##### ROTATE

<div align="center">
    <img src="./report_images/BOMR_Rotation.PNG" alt="ROTATE mode illustration">
    <em>ROTATE mode illustration</em>
</div>


As we observe in the picture, the ROTATE mode calculates the error between the desired angle and the Thymio's actual angle. The Thymio receives the shortest turning direction to align itself with the desired angle, and rotates on its axis to start the correction. Find below the code snippet, taken from the `pi_regulation` method of `MotionControl`'s `Motion` class.
```python
# Movement at Rotate means that the robot need to rotate to face the next point
elif movement == ROTATE:

    if (self._total_wanted_angle - self._total_actual_angle) >= 0:
        if self._total_wanted_angle - self._total_actual_angle >= 180:
            left_speed = self.normal_speed
            right_speed = -self.normal_speed
        else:
            left_speed = -self.normal_speed
            right_speed = self.normal_speed
    elif (self._total_wanted_angle - self._total_actual_angle) < 0:
        if (self._total_actual_angle - self._total_wanted_angle) > 180:
            left_speed = -self.normal_speed
            right_speed = self.normal_speed
        else:
            left_speed = self.normal_speed
            right_speed = -self.normal_speed
            
    return left_speed, right_speed
```

When the Thymio has reached the correct angle, (either as signaled by the camera, or estimated by Kalman filtering), the Thymio will execute the MOVE command towards the next point.

Find below the code snippet, aslo from `pi_regulation`, deciding on state transition:
```python
 if (change_idx != self.change_idx and abs(self._total_actual_angle - self._total_wanted_angle) > TRANSITION_ANGLE):
            movement = ROTATE
        elif change_idx != self.change_idx:
            self.change_idx = change_idx
        elif abs(self._total_wanted_angle - self._total_wanted_angle) > CHANGE_STATE:
            self.change_idx = -1
            movement = ROTATE
```
Note that `total_angle`s are a tracking of the Thymio's orientation unbounded by 0-360° angle constraints which otherwise cause large, sudden spikes in error when the Thymio is oriented along the 0° axis.

<a id="move"></a> 
##### MOVE
<div align="center">
    <img src="./report_images/BOMR_PI.PNG" alt="MOVE Mode illustration">
    <em>MOVE Mode illustration</em>
</div>


The PI controller handles the Thymio's alignement during its movement towards the next point: it works to maintain the Thymio's direction inside of the threshold angle. When the Thymio is in the threshold zone, *the green cone*, both wheels rotate at the same velocity, it moves straight ahead. When the Thymio's orientation exits the tolerated cone, the PI controller corrects the direction by increasing the speed of one wheel and decreasing the speed of the other. The regulation works until the error is inside the threshold zone, in order to avoid oscillation around the desired angle.
     
As explained before, the PI controller is active during the robot's movement, except when it enters obstacle avoidance, managed by *Local Navigation*. Our regulator parameters, *Kp* and *Ki*, were tuned experimentally, by observing how the robot corrected its error with small gains, and progressively increased the values of our gains to achieve a smooth and dynamic correction. Finally, we obtanied satisfactory results with a *Kp = 3* and a *Ki = 0.2*.

We implemented a parameter, called *sum_error_max* to avoid wheel drift during the movement, from integrator error saturation. The maximum value was determined experimentally, similar to the regulator parameters, to mix good dynamics and avoid overshoot and jerky behaviour from integral windup. This is also provided below as a snippet from `pi_regulation`:

```python
# movement at Move means that the robot is achieving the next point
if movement == MOVE:
    left_speed = STOP_MOVING
    right_speed = STOP_MOVING

    # Control if the Thymio is inside the threshold cone of acceptance. If it's not the case
    # the pi controller is active to correct the trajectory
    if ((self._total_wanted_angle - self.threshold_angle) <= self._total_actual_angle
            and self._total_actual_angle <= (self._total_wanted_angle + self.threshold_angle)):
        left_speed = self.normal_speed
        right_speed = self.normal_speed

        self.error = 0
        self.sum_error = 0
    
    elif self._total_actual_angle < (self._total_wanted_angle - self.threshold_angle):
        self.error = abs(self._total_actual_angle - self._total_wanted_angle)
        if self.error > 330:
            self.error = 360 - self.error

        self.sum_error += self.error

        # To avoid that the Thymio start to drift with high speed in a wheel, we implemented an anti wind-up
        # to protect from the integrator of our controller
        if self.sum_error < self.max_sum_error:
            left_speed = self.normal_speed - (self.Kp * self.error + self.Ki * self.sum_error)
            right_speed = self.normal_speed + (self.Kp * self.error + self.Ki * self.sum_error)

        else:
            left_speed = self.normal_speed - (self.Kp * self.error + self.Ki * self.max_sum_error)
            right_speed = self.normal_speed + (self.Kp * self.error + self.Ki * self.max_sum_error)

    elif self._total_actual_angle > (self._total_wanted_angle + self.threshold_angle):
        self.error = abs(self._total_actual_angle - self._total_wanted_angle)
        self.sum_error += self.error

        # To avoid that the Thymio start to drift with high speed in a wheel, we implemented an anti wind-up
        # to protect from the integrator of our controller
        if self.sum_error < self.max_sum_error:
            left_speed = self.normal_speed + (self.Kp * self.error + self.Ki * self.sum_error)
            right_speed = self.normal_speed - (self.Kp * self.error + self.Ki * self.sum_error)

        else:
            left_speed = self.normal_speed + (self.Kp * self.error + self.Ki * self.max_sum_error)
            right_speed = self.normal_speed - (self.Kp * self.error + self.Ki * self.max_sum_error)

    return left_speed, right_speed

```

<a id="movement-schematic"></a> 
##### Schematic of the Movement
Finally, you can observe, with the next schematic, how our Thymio's motion control is configure to perform its mission.
<div align="center">
    <img src="./report_images/BOMR_MotionStep.PNG" alt="Schematic of the Movement">
    <em>Schematic of the Movement</em>
</div>

Our helper functions are defined below, taken `MotionControl`'s `Motion` class:

```python
#Here are some helper functions used for more convenient calls within MotionControl.py, also imported or copied into other files as needed
    def motors(self, speed_left, speed_right):
        """"
        Send the information to provide to the Thymio for changing the speed of the wheels
        Return : "motor.left.target": [int(speed_left)], "motor.right.target": [int(speed_right)]
        """
        return {
            "motor.left.target": [int(speed_left)], "motor.right.target": [int(speed_right)],
        }

    def move(self, left_speed, right_speed):
        """"
        Send the new velocity of the motors to the Thymio
        """
        aw(self.node.send_set_variables(self.motors(int(left_speed), int(right_speed))))
```

<a id="orientation-angle-challenges"></a> 
##### Orientation Angle Challenges
One problem we encountered was with the orientation computations for Thymio's the path planning. As the angles were given in the range \[0, 360), driving straight down on the map, which is a 0° orientation in our coordinate system was problematic. Whenever there were slight orientation changes while driving, the orientation angle of Thymio would jump from 0° to 360° and vice versa. 

As our algorithm relied on computing the difference between the actual orientation of the Thymio and the wanted orientation, such large changes strongly impacted its ability to drive smoothly. We were able to solve this by keeping track of the previously measured orientation and wanted direction and based the new motion control commands on the differences of the previous and new measurements.

A similar approach was taken in the orientation estimation of our [Kalman Filter](#handling-orientation-jumps) implementation to prevent the camera's orientation estimation from dramatically changing predictions due to such angle jumps.

Find below the relevant code snippet considering angle manipulations from `MotionControl`, also taken from `pi_regulation`:

```python
if abs(actual_angle - self._last_actual_angle) > ANGLE_JUMP_TOLERANCE:
    if actual_angle > self._last_actual_angle:
        actual_angle -= 360
    else:
        actual_angle += 360
if abs(wanted_angle - self._last_wanted_angle) > ANGLE_JUMP_TOLERANCE:
    if wanted_angle > self._last_wanted_angle:
        wanted_angle -= 360
    else:
        wanted_angle += 360
difference_actual_angle = actual_angle - self._last_actual_angle
difference_wanted_angle = wanted_angle - self._last_wanted_angle
self._total_actual_angle += difference_actual_angle
self._total_wanted_angle += difference_wanted_angle
while abs(self._total_actual_angle - self._total_wanted_angle) > 360:
    if self._total_actual_angle > self._total_wanted_angle:
        self._total_actual_angle -= 360
    else:
        self._total_actual_angle += 360
self._last_actual_angle = actual_angle
self._last_wanted_angle = wanted_angle

```

<a id="local-navigation"></a>
### 3.4 Local Navigation

<a id="judge-severity"></a> 

##### Judge Severity

In our project, the Thymio robot's local navigation is intricately linked to the status of its proximity sensors. Since, in our project, the robot does not move backwards, the sensors at the back of the Thymio are ignored. At the core of this functionality is an instance of the `LocalNavigation` class, initialized at program startup, responsible for tracking relevant variables associated with local navigation.

During each iteration of the primary while True loop, the proximity sensors values are pulled, and they are sent to the `judge_severity()` method. This method, determines the danger level for each sensor, which increases when obstacles are nearer. The maximum danger level among these sensors is then assigned to the `danger_state` attribute within the local navigation instance.

The DangerState enum encapsulates threshold values, which have been fine-tuned through experimentation to reflect different danger levels. In cases where the Thymio finds itself in an unsafe state, global navigation is temporarily suspended and a status variable (`local_nav.state`) is modified to track in which phase of local navigation the Thymio is. 


This code is taken from ```LocalNavigation.py```
```python
def judge_severity(self):
    """
    Updates the danger_state of the LocalNavigation object and returns it
    """
    self.danger_state = DangerState.SAFE
    #Do no handle the back sensors, since Thymio does not reverse in Global Nav
    for i in range(5):
        if (self.prox_horizontal[i] > SensorThresh.STOP_THRESH.value):
            self.danger_state = DangerState.STOP
            return self.danger_state    #If any sensor is at STOP, we do not overwrite it
        elif (self.prox_horizontal[i] > SensorThresh.WARN_THRESH.value):
            self.danger_state = DangerState.WARN
    return self.danger_state
```

And this excerpt from ```LocalNavigation.run()```
```python
#Danger
if self.danger_state == DangerState.STOP and self.state == LocalNavState.START:
    self.state = LocalNavState.ROTATING #Start turning
```

Local Navigation unfolds in two distinct phases:

<a id="rotation"></a>
##### Rotation:
In this phase, the Thymio robot determines a turning direction based on the global path's next turn direction. The objective is to navigate along the outer edge of the turn, a strategy particularly effective when following paths generated by Dijkstra navigation. As these paths tend to adhere closely to global obstacles, adopting an outward trajectory facilitates global obstacle avoidance.

    Once the turning direction is determined, the Thymio initiates a rotational movement in place. This rotation persists until the robot no longer detects obstacles in its proximity. Importantly, even after achieving clearance, the robot continues to rotate for a few additional iterations. This deliberate extension adds a margin of safety, preventing the Thymio from executing a turn too close to an obstacle and potentially colliding with it.

```python
def determine_turn_dir(self, dir_changes, next_dir_change_idx):
    """
    Determines turn direction for circling an object, based on the Dijktra path.
    Since the path tends to stick to object, circling outside the next turn will
    avoid the Local obstacle without crossing a global obstacle.
    Updates turn_dir, to be called in turn()
    """
    previous_cell = np.array(dir_changes[next_dir_change_idx-1])
    current_cell = np.array(dir_changes[next_dir_change_idx])
    next_cell = np.array(dir_changes[next_dir_change_idx+1])

    current_vector = current_cell - previous_cell
    next_vector = next_cell - current_cell

    orientation_current_v = rotation_nextpoint(current_vector)
    orientation_next_v = rotation_nextpoint(next_vector)

    orientation_difference = (orientation_next_v - orientation_current_v)%360
    #We have now the turn angle of the next turn in the path, determine which way to turn to circle wide around it

    if orientation_difference < 180:
        self.turn_dir = 'r'
    else:
        self.turn_dir = 'l'
```

These `'r'` and `'l'` variables are placeholders to signify whether the Thymio needs to circle to the right or to the left, as can be seen in the translator function, `LocalNavigation.turn()`, presented here:

```python
def turn(self):
    """
    Reads a direction from the turn_dir field
    Call this to turn left with 'l', right with 'r'
    """
    if self.dir == 'r':
        motor_right_target = -NORMAL_SPEED
        motor_left_target = NORMAL_SPEED
    elif self.dir == 'l':
        motor_right_target = NORMAL_SPEED
        motor_left_target = -NORMAL_SPEED
    else:   #failsafe, do nothing
        motor_right_target = 0
        motor_left_target = 0
    return motor_left_target, motor_right_target
```

<a id="circling"></a>
##### Circling:
With the Thymio's orientation now firmly established through the preceding rotation phase, the robot enters the circling phase. In this stage, forward motion is initiated with a subtle offset between the left and right motors, enabling the Thymio to trace a wide circle around the detected obstacle.

If an obstacle is detected during this circling motion, the robot seamlessly transitions back to the turning phase, reverting to the initiallychosen turning direction. This strategic continuity allows the Thymio to gracefully navigate around elongated obstacles by tracing circles along their edge.

Again, this code is presented form `LocalNavigation.run()`

```python
if self.state == LocalNavState.CIRCLING:
    self.circle_counter += 1       #Counter for circling, gives some slack to the thymio to avoid staying in place
    
    if self.danger_state != DangerState.SAFE:   #Encounters another obstacle, or circle was too tight
        motor_speeds = self.turn()
        self.rotate_counter = 0  
        self.circle_counter = 0     #Reset counters, we need to start over
        return motor_speeds
    
    #We can circle safely
    forward_speed = NORMAL_SPEED - SPEED_OFFSET      #Fastest wheel turns at normal speed 
    if self.turn_dir == 'l':
        left_coeff = 1
        right_coeff = -TURN_COEFF
    else:
        left_coeff = -TURN_COEFF
        right_coeff = 1
    left_speed = forward_speed + left_coeff*SPEED_OFFSET
    right_speed= forward_speed +right_coeff*SPEED_OFFSET
    return left_speed, right_speed
return motor_speeds
```

The circling phase concludes when the Thymio reconnects with the Global Navigation path, which is bound to happen, as pathfinding is designed to create non-looping paths. Once the Thymio detects its return to the path, it recomputes the remaining checkpoints (turning points) ahead, selects the next one in line, and gracefully relinquishes control back to Global Navigation and Motion Control, to steer the Thymio forth to the goal.

This code snippet is taken from the `Main.py`'s `while` loop:

```python
resume_path_cell = dijkstra.map.check_if_returned_to_path()
if (danger_level != DangerState.STOP and
    (local_nav.circle_counter > CIRCLE_COUNTER_BUFFER and resume_path_cell is not None)):  # Back to path, wtih some slack
    # Done with local nav
    local_nav.reset_state()
    dijkstra.handle_local_navigation_exit(thymio_location=resume_path_cell)
```

This method is taken from `GlobalNavigation`

```python
def handle_local_navigation_exit(self, thymio_location):
    """
    Handles the exit of local navigation and takeover of global navigation
    :param thymio_location: The Thymio's location
    :return: None
    """
    x_thymio, y_thymio = thymio_location
    self._next_direction_change_idx = 0
    x_dir_change, y_dir_change = self.map.direction_changes[self._next_direction_change_idx]
    for (x_path, y_path) in self.path:
        if x_path == x_dir_change and y_path == y_dir_change:
            self._next_direction_change_idx += 1
            x_dir_change, y_dir_change = self.map.direction_changes[self._next_direction_change_idx]
        if x_path == x_thymio and y_path == y_thymio:
            break
```

Local Navigation therefore acts as a simple Finite State Machine, illustrated in the image below:

<div align="center">
    <img src="./report_images/image_processing_images/local_navigation_fsm.png">
    <em>Local Navigation Finite State Machine illustration</em>
</div>



The START State allows Global Navigation to steer the Thymio freely, so long as no obstacle is detected by the sensors, which will drive a state change.

If the Thymio's marker is obscured during Local Navigation, it will still move using the Kalman Filter to estimate position. Once sight is restored, if the Thymio's position has changed too much, the Thymio will recompute the optimal path and use that to move. This is necessary, as it allows us to kidnap the Thymio during Local Navigation, and have it handle well. This situation also happens in the case of tall obstacles, and typically the Thymio's new path will not cross the obstacle, but even if it does, circling will work well as it will not go back behind the obstacle.

*The counters on the Thymio's rotation are staggered by Global Navigation's `wait_for_variables`. While this is not exact, we do not need an exact delay as we are not aiming for perfect navigation, we just need a little bit of slack in this application!

The local navigation path is illustrated below:

<div align="center">
    <img src="./report_images/image_processing_images/grid_map_with_local_avoidance.png">
    <em>Illustration of Local Avoidance Path</em>
</div>

The Thymio detects the object (here in blue) and deviates from the Global Path (in red), choosing to go on the *outside* of the path's turn, thus drawing a circle, here in orange.

Please note, however, that this image is never generated, as the camera and the computer are never aware of the local obstacle, this is only an illustration of a path possibly taken during local navigation.


<a id="global-navigation-resumption"></a>

##### Global Navigation Resumption
Once the Thymio arrives back on the path, global navigation takes over again. In order to account for the distance travelled, first the location of the Thymio on the path is found and, then, the intermediate goals are updated to account for having passed a variable number of them during local obstacle avoidance. This is handled in the following function in the ```DijkstraNavigation``` class:
```Python
def handle_local_navigation_exit(self, thymio_location):
    """
    Handles the exit of local navigation and takeover of global navigation
    :param thymio_location: The Thymio's location
    :return: None
    """
    x_thymio, y_thymio = thymio_location

    self._next_direction_change_idx = 0
    x_dir_change, y_dir_change = self.map.direction_changes[self._next_direction_change_idx]
    for (x_path, y_path) in self.path:
        if x_path == x_dir_change and y_path == y_dir_change:
            self._next_direction_change_idx += 1
            x_dir_change, y_dir_change = self.map.direction_changes[self._next_direction_change_idx]

        if x_path == x_thymio and y_path == y_thymio:
            break
```

<a id="kalman-filter"></a>
### 3.5 Kalman Filter

Our Kalman filter implementation serves a critical role in the position and orientation estimation of our Thymio. It uses the KalmanFilter library of pythons filterpy package and combines the wheel motor speed of the Thymio with the information provided by the computer vision part to increase accuracy.

<a id="wheel-speed-measurements"></a>
##### Wheel Speed Measurements
The wheel speed measurements are used for both estimating the position and orientation of the Thymio. A refinement was introduced by incorporating a max-min constraint, to mitigate the impact of noise on the speed sensors. This addition serves to limit the potential noise-induced variations in the sensor readings, contributing to a more stable and reliable interpretation of the Thymio robot's speed information. To predict the new orientation, the angular velocity is calculated based on the difference of wheel speeds and a baseline minimum period for a 360° turn of the Thymio. 

```python
# WHEEL_MIN_SPEED <= v <= WHEEL_MAX_SPEED
v = (left_wheel_speed + right_wheel_speed) / 2.0

right_wheel_speed = max(min(right_wheel_speed, WHEEL_MAX_SPEED), WHEEL_MIN_SPEED)
left_wheel_speed = max(min(left_wheel_speed, WHEEL_MAX_SPEED), WHEEL_MIN_SPEED)

period = 0
if abs(right_wheel_speed - left_wheel_speed) > 0.5:  # avoid division by 0 and large values
    period = ((2*WHEEL_MAX_SPEED) / (right_wheel_speed - left_wheel_speed)) * MIN_THYMIO_360_TURN_PERIOD  # positive sense is counterclockwise
    w = 0
    if period != 0:
        w = - 2 * np.pi / period
```

##### Camera input
As the camera input already is provided in grid coordinates by the computer vision, there is no additional processing needed for it.

<a id="prediction-update-step"></a>
##### Prediction and Update Step
We use the wheel speed measurements to perform the prediction step of the Kalman filter and use the computer vision information in the update method. With the combination of these two informations, the Kalman filter provides an accurate estimation of the position and orientation of the Thymio robot.


<a id="handling-orientation-jumps"></a>
 ##### Handling Orientation Jumps
As the provided orientation input from our computer vision module is a float between 0 and 2π, depending on the Thymio's orientation, there may be jumps in the estimated orientation from 0 to 2π and vice versa. This leads to large jumps in the overall estimation of the orientation. To combat that, we do not limit the predicted orientation to \[0, 2π) but let it be any number and scale using the mod operation only when the navigation part accesses it. This dramatically improved the orientation estimation of our filter.

```Python

# always add the change in angle from last time to this time to the "total" angle
angle_change = direction_angle_camera - self.last_temp_direction_angle_camera

# since only small angle changes expected, we can assume that the change is between -ANGLE_JUMP_TOLERANCE and ANGLE_JUMP_TOLERANCE
# --> if not, we need to wrap around (as there is a jump from 360 to 0 degrees or vice versa)
if angle_change > ANGLE_JUMP_TOLERANCE:
    angle_change = angle_change - 2 * np.pi
elif angle_change < -ANGLE_JUMP_TOLERANCE:
    angle_change = angle_change + 2 * np.pi

self.total_direction_angle_camera = self.total_direction_angle_camera + angle_change
self.last_temp_direction_angle_camera = direction_angle_camera
        
```

<a id="adaptability-vision"></a>
##### Adaptability to Vision Feed Cut-Off
Whenever the camera feed is unavailable, the Kalman filter continues to estimate the postition and orientation based solely on the predictions derived from the wheel speed measurements. This allows the Thymio to maintain knowledge of its position and orientation even when external measurements are unavailable

The Kalman filter's ability to combine information from wheel speed measurements and vision data drastically increases the robustness of our navigation and even allows for accurate driving when no computer vision data is available.


##### Details in the choice of matrices 
the matrices for the kalman filter are the following:
- F = State Transition Matrix
- Q = Process Noise Covariance
- H = Measurement Function  
- R =  Measurement Noise Covariance

\begin{aligned}
& F=\left[\begin{array}{llll}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{array}\right] \quad Q=\left[\begin{array}{llll}
\sigma_{Qx}^2 & 0 & 0 & 0 \\
0 & \sigma_{Qy}^2 & 0 & 0 \\
0 & 0 & \sigma_{Q\theta}^2 & 0 \\
0 & 0 & 0 & 0
\end{array}\right]  \\
& H=\left[\begin{array}{llll}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{array}\right] 
\quad R=\left[\begin{array}{llll}
\sigma_{Rx}^2 & 0 & 0 & 0 \\
0 & \sigma_{Ry}^2 & 0 & 0 \\
0 & 0 & \sigma_{R\theta}^2 & 0 \\
0 & 0 & 0 & 0.1
\end{array}\right] 
\end{aligned}


The variances were determined through experimental considerations. The best values were found to be:
> $\sigma_{Qx}^2 = \sigma_{Qy}^2 = 0.001, \ \sigma_{Q\theta}^2 = 1$

> $\sigma_{Rx}^2 = \sigma_{Ry}^2 = \sigma_{R\theta}^2 = 0.1$
 

Each parameter was tuned seperately under the assumption that there is no correlation between the parameters.

<a id="results"></a>
### 4. Results




The image processing and vision components demonstrated robustness, consistently delivering precise results in obstacle detection, Thymio robot identification, and goal localization. In particular, the implementation of live camera feed utilizing ArUco markers was particularly precise, even enabling successful recovery from simulated kidnapping scenarios.

In terms of navigation, the Dijkstra algorithm efficiently derived optimized paths, allowing the Thymio robot to traverse from the starting point to the goal with notable efficiency. The implementation of Local Navigation further enhanced the system's capabilities, enabling it to detect and navigate around obstacles not visible to the camera while maintaining a path to the goal.

The primary challenge confronted by the system involved accurately representing angles within the [0, 360) range, for both filtering and motion control. The orientation is represented as a floating-point value between 0 and 2π (0 to 360 degrees). Consequently, due to the Thymio's orientation, there is a potential for discontinuities or jumps in the estimated angle values.

To address this issue in filtering, we opted for a solution that involved not restricting the orientation to a specific range but allowing it to be any number and then scaling it using the mod operation, effectively preventing jumps and maintaining a continuous representation.

In the context of motion control, a successful remedy was implemented by introducing a motion control mechanism that considers both the previous and new orientation estimations. This approach has proven effective in mitigating the problem, significantly enhancing the overall performance of the system.

<a id="video"></a>
##### Video: 
The following video illustrates the combination of the five main sections of this project. To illustrate the knowledge of the program, the right video feed shows the grid with all the elements. Additionally, the wanted current direction as well as the Thymio's current orientation are drawn on the map image.

The left video feed shows the recording made by the camera with the transformation matrix applied. When the camera is covered by a paper, the locaiton and orientation are estimated by the Kalman filter based on the wheel speeds.

<div align="center">
    <video width="700" controls autoplay loop src="./videos/Mobile_Robotics_Final_Demo_Video.mp4" />
</div>

<div align="center">
    <em>Video Illustrating the Project Results</em>
</div>

In case the video does not play (support issues for certain kernels), it can also be watched on [YouTube](https://youtu.be/gpMigWjB38A)

<a id="demo"></a>
# Run Demo

In [ ]:
%run ./Main.py

<a id="references"></a>
### Resources 

- OpenCv ArUco Markers: https://docs.opencv.org/3.2.0/d5/dae/tutorial_aruco_detection.html
- ArUco Marker Generation: https://chev.me/arucogen/
- Kalman Filter: https://filterpy.readthedocs.io/en/latest/kalman/KalmanFilter.html



All images included in this report are our own work, either screenshots of opencv2 windows, made using Blender, Adobe Photoshop or the tikz library in LaTex.